In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import numpy as np
import requests
from talib import MACD, RSI, BBANDS
from datetime import datetime, timedelta, date
from IPython.display import display, HTML
%matplotlib inline
import json
import math
from dateutil import parser
from pytz import timezone
import pytz
import sys
import iso8601
from pandas import DataFrame as df

In [ ]:
api = tradeapi.REST(base_url="https://api.alpaca.markets")
session = requests.session()

In [ ]:
start_day_to_analyze = '2020-07-20'
end_day_to_analyze = '2020-07-21'
symbols = ['SNAP']

In [ ]:
minute_history = {}

for symbol in symbols:
    if symbol not in minute_history:
        minute_history[symbol] = api.polygon.historic_agg_v2(
            symbol, 
            1, 
            'minute',
            _from = str((datetime.strptime(start_day_to_analyze, '%Y-%m-%d')-timedelta(days=10)).date()),
            to=str((datetime.strptime(end_day_to_analyze, '%Y-%m-%d')+timedelta(days=1)).date())).df
        print(f"{symbol} loaded {len(minute_history[symbol].index)} agg data points")


In [ ]:
def grouper(iterable):
    prev = None
    group = []
    for item in iterable:

        if not prev or -0.02 <= float(item - prev) / prev <= 0.02:
            group.append(item)
        else:
            yield group
            group = [item]
        prev = item
    if group:
        yield group


def find_resistance(current_value: float, minute_history, now):
    """calculate next resistance"""
    est = pytz.timezone("US/Eastern")
    now = pd.Timestamp(now.astimezone(est))
    minute_history_index = minute_history["close"].index.get_loc(now, method="nearest")
    back_time = pd.Timestamp(now.to_pydatetime() - timedelta(days=4))
    back_time_index = minute_history["close"].index.get_loc(back_time, method="nearest")
    for back_track_min in range(
        minute_history_index + 1 - 120, back_time_index, -5
    ):  
        series = (
            minute_history["close"][back_track_min : minute_history_index + 1]
            .dropna()
            .between_time("9:30", "16:00")
            .resample("15min")
            .max()
        ).dropna()
        diff = np.diff(series.values)

        high_index = np.where((diff[:-1] >= 0) & (diff[1:] <= 0))[0] + 1
        if len(high_index) > 0:
            local_maximas = sorted(
                [series[i] for i in high_index if series[i] >= (current_value)]
            )
            if len(local_maximas) > 0:
                return local_maximas

    return None


def find_support(current_value, minute_history, now):
    """calculate support"""
    est = pytz.timezone("US/Eastern")
    now = pd.Timestamp(now.astimezone(est))
    print(f"find_support({now})")
    minute_history_index = minute_history["close"].index.get_loc(now, method="nearest")
    back_time = pd.Timestamp(now.to_pydatetime() - timedelta(days=4))
    back_time_index = minute_history["close"].index.get_loc(back_time, method="nearest")
    for back_track_min in range(
        minute_history_index + 1 - 120, back_time_index, -5
    ):  # len(minute_history[:minute_history_index+1].index),
        # print("start")
        series = (
            minute_history["close"][back_track_min : minute_history_index + 1]
            .dropna()
            .between_time("9:30", "16:00")
            .resample("5min")
            .max()
        ).dropna()
        diff = np.diff(series.values)
        high_index = np.where((diff[:-1] <= 0) & (diff[1:] > 0))[0] + 1
        if len(high_index) > 0:
            local_maximas = sorted(
                [series[i] for i in high_index if series[i] <= current_value]
            )
            if len(local_maximas) > 0:
                # print(minute_history[:minute_history_index+1])
                # print(minute_history["close"][minute_history_index], series)
                return local_maximas

    return None

In [ ]:
for symbol in minute_history:
    start_date = datetime.strptime(start_day_to_analyze, "%Y-%m-%d")
    start_date = start_date.replace(hour=9, minute=30)
    end_date = start_date.replace(hour=16, minute=0)
    start_index = minute_history[symbol]["close"].index.get_loc(
        start_date, method="nearest"
    )
    start_end = minute_history[symbol]["close"].index.get_loc(
        end_date, method="nearest"
    )

    minute_history[symbol]["resistance"] = minute_history[symbol][
        start_index:start_end
    ].apply(
        lambda x: val[0]
        if (val := find_resistance(x["close"], minute_history[symbol], x.name))
        is not None
        else None,
        axis=1,
    )

In [ ]:
for symbol in minute_history:
    start_date = datetime.strptime(start_day_to_analyze, "%Y-%m-%d")
    start_date = start_date.replace(hour=9, minute=30)
    print(start_date)
    minute_history_index = minute_history[symbol]["close"].index.get_loc(
        start_date, method="nearest"
    )
    plt.plot(
        minute_history[symbol]["close"][minute_history_index:].between_time(
            "9:30", "16:00"
        ),
        label=symbol,
    )
    plt.xticks(rotation=45)

    resistances = minute_history[symbol].resistance.dropna().unique().tolist()
    if resistances:
        for r in resistances:
            plt.axhline(y=r, color="r")
    plt.legend()
    plt.show()